In [61]:
import aop
import aop.api
import json

import pymysql
import pandas as pd
import numpy as np
import datetime
import time

#google sheet 專用   #pip install google-api-python-client
import string
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [62]:
# 時間日期設定
today='2022-09-21'#(datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d") 
# 设置网关域名
aop.set_default_server('gateway.open.umeng.com')
# 设置apiKey和apiSecurity
aop.set_default_appinfo(6560937, "icwdcKfDDGY")
# appkey_channel
appkey_h5_new='6320305405844627b54716ca'

In [63]:
# 构造Request搜尋總計
req1 = aop.api.UmengUminiGetEventOverviewRequest()
resp1 = req1.get_response(None, timeUnit="day", fromDate=today, toDate=today, eventName="search_total", dataSourceId=appkey_h5_new)

# 构造Request成功搜尋次數
req2 = aop.api.UmengUminiGetEventOverviewRequest()
resp2 = req2.get_response(None, timeUnit="day", fromDate=today, toDate=today, eventName="search_success", dataSourceId=appkey_h5_new)

In [64]:
resp1_evevt=pd.DataFrame(resp1['data']['data'])
resp1_evevt['eventname']='搜尋總計'
resp1_evevt=resp1_evevt[['dateTime','eventname','count','device']]

resp2_evevt=pd.DataFrame(resp2['data']['data'])
resp2_evevt['eventname']='成功搜尋次數'
resp2_evevt=resp2_evevt[['dateTime','eventname','count','device']]

#合併事件
dd_event=resp1_evevt.append(resp2_evevt)

In [65]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
      'https://www.googleapis.com/auth/drive',
      'https://www.googleapis.com/auth/spreadsheets']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()

#寫入 data 進入 google sheet #美東時間
def insert_data(value_range_body,sheetname,colname1,colname2,col1,col2) :
    SAMPLE_RANGE_NAME = str(sheetname)+'!'+colname1+str(col1)+':'+colname2+str(col2)
    request= sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,valueInputOption='USER_ENTERED',range=SAMPLE_RANGE_NAME , body=value_range_body).execute()
def google_sheet_insert_data(sheet_name,data,SAMPLE_SPREADSHEET_ID) : 
    #查詢目前使用到維度
    SAMPLE_RANGE_NAME = sheet_name+'!A1:A60000'
    result_index = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    insert_row=len(result_index.get('values'))+1
    #遊戲驗證
    value_range_body = {"majorDimension":"ROWS","values": data.values.tolist() }
    insert_data(value_range_body,sheet_name,'A','D',insert_row,insert_row+10)

In [66]:
SAMPLE_SPREADSHEET_ID = '1nVa7_RQ3QIxzepd3usKqm5BU7C567L4Sb5ZzoZCCmV8'
google_sheet_insert_data('友盟VIP数据',dd_event,SAMPLE_SPREADSHEET_ID)